In [151]:
import random
import numpy as np
import os

print("testing...")

testing...


In [152]:
# Consider discretizing the labels so the arrays can be numpy
# DATASET.PARTITIONS FORMAT: dataset.partitions[fold][example][attribute]
'''
self.line_count - total # of examples
self.partition_count - total # of partitions
self.total_example_count - total # of examples
self.attribute_count - total # of attributes for each example
'''

class dataset:
    def __init__(self, data_path, label_location, processed_flag):
        if (processed_flag == False):
            # Separating the .data file into lines, and shuffling the lines
            with open(data_path, 'r') as file:
                lines = file.readlines()
            random.shuffle(lines)
            self.data_lines = lines
            #print(f"Data Lines Array Unprocessed: {self.data_lines}")

            # Deliminate strings into lists
            for i in range(len(self.data_lines)):
                self.data_lines[i] = self.data_lines[i].strip()
                self.data_lines[i] = self.data_lines[i].split(',')

            # If the labels are in the first location, move them to the last location in the line
            #print(f"Data Lines Array Deliminated: {self.data_lines}")
            if (label_location == 'first'):
                for i in range(len(self.data_lines)):
                    first_item = self.data_lines[i].pop(0)
                    self.data_lines[i].append(first_item)
                #print(f"Data Lines Array W/ Indices Swapped: {self.data_lines}")

            # Folding the data...
            self.line_count = len(self.data_lines)
            partition_size = self.line_count//10
            partition_remainder = self.line_count%10
            self.partitions = [self.data_lines[i * partition_size:(i + 1) * partition_size] for i in range(10)] # Separating the lines list into 10 lists of lines
            if (partition_remainder != 0):
                for i in range(partition_remainder):
                    # MIGHT NEED TO ADD EXTRA INDICE DIMENSION WHEN DATA IS DELIMINATED PRIOR
                    self.partitions[:][i].append(self.data_lines[-(i+1)])
        else:
            #extract processed data
            self.partitions = []
            with open(data_path, 'r') as file:  #open csv file of processed data
                for line in file:
                    line = line.strip() #strip each line of any white space or newlines
                    examples = line.split(";") #Split each line into examples by each semicolon
                    attributes = [list(map(str, item.split(","))) for item in examples] #seperate each attribute and remove commas
                    self.partitions.append(attributes) # put into 3d array

        # General dataset information setters
        self.partition_count = 10
        #self.total_example_count = self.line_count
        self.attribute_count = len(self.partitions[0][0])

    def save(self, save_file_name, save_folder):
        # Saves the data based on our convention: Each line is a partition, semicolons separate examples, commas separate attributes/labels
        folder_path = os.path.expanduser(f"{save_folder}/processed_data")  
        os.makedirs(folder_path, exist_ok=True)
        #get/create the path to the folder that the file should be saved to
        file_path = os.path.join(folder_path, save_file_name)
        #create the file path
        with open(f"{file_path}.csv", "w") as file:
            #open a csv file in the desired location
            for line in self.partitions:
                partition_lines = ";".join([",".join(map(str, sub_array)) for sub_array in line])
                #for each partition, join each example by a semi colon and each attribute by a comma
                file.write(partition_lines + "\n")
                #write each partition into the file with each 
        print(f"CSV file saved to {file_path}")
        return

    def imputate(self, voter_bool):
        # Replaces question marks in a dataset with a random value between the min/max of an attribute value
        # Breast cancer has a range of 1-10 for the attribute that is missing values
        # voter_bool indicates whether you are working with the voter dataset, where attributes are strings of 'y' or 'n' instead of numbers
        voter_options = ['y', 'n']
        
        for partition in range(len(self.partitions)):
            for example in range(len(self.partitions[partition])):
                for attribute in range(len(self.partitions[partition][example])):
                    # if this statement is entered that means there is a missing piece of attribute data, so imputation needs to occur at this location
                    if (self.partitions[partition][example][attribute] == '?'):
                        # This will be the imputation method using 'y' or 'n'
                        if (voter_bool == True):
                            self.partitions[partition][example][attribute] = random.choice(voter_options)
                        # This will be the imputation method using range 1-10
                        else:
                            self.partitions[partition][example][attribute] = str(random.randint(1,10))



    def discretize(self, divisions, glass_iris):
        # divisions: how much you want the bins to be divided by
        # glass_iris: boolean that indicates whether you are working with the glass or iris dataset. True = glass, False = Iris
        glass_extrema = [[1.5112, 1.5339],[10.73,17.38],[0,4.49],[0.29,3.5],[69.81,75.41],[0,6.21],[5.43,16.19],[0,3.15],[0,0.51]]
        iris_extrema = [[4.3,7.9],[2.0,4.4],[1.0,6.9],[0.1,2.5]]
        glass_ranges = []
        iris_ranges = []
        # Creates the increment level for the bin ranges
        for extrema in glass_extrema:
            glass_ranges.append(round((extrema[1]-extrema[0])/divisions,6))
        for extrema in iris_extrema:
            iris_ranges.append(round((extrema[1]-extrema[0])/divisions,2))

        
        # Takes a continous attribute and "bins" the attribute into discrete groups. May need to create a dictionary
        for partition in range(len(self.partitions)):
            for example in range(len(self.partitions[partition])):
                for attribute in range((len(self.partitions[partition][example])-1)):
                    # Run glass discretization
                    if (glass_iris == True):
                        entered = False
                        for i in range(divisions):
                            if ((float(self.partitions[partition][example][attribute]) >= (glass_extrema[attribute][0] + (glass_ranges[attribute] * i))) and (float(self.partitions[partition][example][attribute]) < (glass_extrema[attribute][0] + (glass_ranges[attribute] * (i+1))))):
                                self.partitions[partition][example][attribute] = str(i)
                                entered = True
                        if entered == False:
                            self.partitions[partition][example][attribute] = str(int(float(self.partitions[partition][example][attribute])))
                    # Run Iris discretization
                    else:
                        entered = False
                        for i in range(divisions):
                            if ((float(self.partitions[partition][example][attribute]) >= (iris_extrema[attribute][0] + (iris_ranges[attribute] * i))) and (float(self.partitions[partition][example][attribute]) < (iris_extrema[attribute][0] + (iris_ranges[attribute] * (i+1))))):
                                self.partitions[partition][example][attribute] = str(i)
                                entered = True
                            if entered == False:
                                self.partitions[partition][example][attribute] = str(int(float(self.partitions[partition][example][attribute])))
                            

    def add_noise(self):
        #selects 10% of the features at random and shuffles the values within each feature, thus introducing noise into the data
        num_to_shuffle = max(1, int(round(0.1 * (self.attribute_count- 1)))) #get the numbr of attributes to shuffle based on 10% of features bing shuffled
        print(f"{self.attribute_count}")
        attribute_noise = random.sample(range(self.attribute_count - 1), num_to_shuffle)# randomly choose an attribute to shuffle
        print(f"Noise Index: {attribute_noise}")
        print(f"No Noise: {self.partitions}" + "\n")
        for attribute_indice in attribute_noise: #loop through each attribute to shuffle in each example and partition
            attributes_values = []
            for partition in self.partitions:
                for example in partition:
                    attributes_values.append(example[attribute_indice]) #store attribute values in a list
            
            random.shuffle(attributes_values) #shuffle selected attribute values
            
            i = 0
            for partition in self.partitions:  #loop through each partition and example
                    for example in partition:
                        example[attribute_indice] = attributes_values[i] #put the attribute value back into the data after being shuffled
                        i += 1
    
    def remove_attribute(self, indice=0):
        # Takes in an attribute indice, and removes that entire indice from the dataset. This can be used to remove ID numbers
        for partition in range(len(self.partitions)):
            for example in range(len(self.partitions[partition])):
                    self.partitions[partition][example].pop(0)

    
        


In [153]:
# dataset instantiation
iris_dataset = dataset('/home/carlos/Machine_Learning_Practice/datasets/iris.data', 'last', False)

In [154]:
# Constructor Testing

print(f"Folded Array: {iris_dataset.partitions}")
print(f"Data first layer length (Should be length 10): {len(iris_dataset.partitions)}")
#print(f"Data second layer length (Should be {iris_dataset.line_count}): {len(iris_dataset.partitions[1])}")
#print(f"Data third layer length (Should be # of items in a line): {len(iris_dataset.partitions[0][0])}")

total_line_count = 0
for i, partition in enumerate(iris_dataset.partitions):
    print(f'Fold {i+1} Length: {len(partition)}')
    total_line_count += len(partition)
print(f"Number of Examples: {total_line_count}")

#print(iris_dataset.partitions[0][0][2])
print(iris_dataset.attribute_count)

Folded Array: [[['6.2', '2.8', '4.8', '1.8', 'Iris-virginica'], ['7.4', '2.8', '6.1', '1.9', 'Iris-virginica'], ['7.7', '2.8', '6.7', '2.0', 'Iris-virginica'], ['5.4', '3.4', '1.5', '0.4', 'Iris-setosa'], ['5.1', '3.4', '1.5', '0.2', 'Iris-setosa'], ['4.9', '2.5', '4.5', '1.7', 'Iris-virginica'], ['5.5', '2.4', '3.8', '1.1', 'Iris-versicolor'], ['6.0', '2.7', '5.1', '1.6', 'Iris-versicolor'], ['6.3', '3.4', '5.6', '2.4', 'Iris-virginica'], ['5.6', '2.5', '3.9', '1.1', 'Iris-versicolor'], ['5.9', '3.2', '4.8', '1.8', 'Iris-versicolor'], ['5.6', '2.7', '4.2', '1.3', 'Iris-versicolor'], ['5.9', '3.0', '4.2', '1.5', 'Iris-versicolor'], ['4.9', '3.0', '1.4', '0.2', 'Iris-setosa'], ['5.9', '3.0', '5.1', '1.8', 'Iris-virginica'], ['6.1', '3.0', '4.6', '1.4', 'Iris-versicolor']], [['6.6', '3.0', '4.4', '1.4', 'Iris-versicolor'], ['5.2', '3.4', '1.4', '0.2', 'Iris-setosa'], ['6.9', '3.1', '5.1', '2.3', 'Iris-virginica'], ['5.8', '2.6', '4.0', '1.2', 'Iris-versicolor'], ['5.8', '2.7', '3.9', '1.

In [155]:
print(f"Un-Discretized Iris Dataset: {iris_dataset.partitions}")
iris_dataset.discretize(10, False)
print(f"Discretized Iris Dataset: {iris_dataset.partitions}")


Un-Discretized Iris Dataset: [[['6.2', '2.8', '4.8', '1.8', 'Iris-virginica'], ['7.4', '2.8', '6.1', '1.9', 'Iris-virginica'], ['7.7', '2.8', '6.7', '2.0', 'Iris-virginica'], ['5.4', '3.4', '1.5', '0.4', 'Iris-setosa'], ['5.1', '3.4', '1.5', '0.2', 'Iris-setosa'], ['4.9', '2.5', '4.5', '1.7', 'Iris-virginica'], ['5.5', '2.4', '3.8', '1.1', 'Iris-versicolor'], ['6.0', '2.7', '5.1', '1.6', 'Iris-versicolor'], ['6.3', '3.4', '5.6', '2.4', 'Iris-virginica'], ['5.6', '2.5', '3.9', '1.1', 'Iris-versicolor'], ['5.9', '3.2', '4.8', '1.8', 'Iris-versicolor'], ['5.6', '2.7', '4.2', '1.3', 'Iris-versicolor'], ['5.9', '3.0', '4.2', '1.5', 'Iris-versicolor'], ['4.9', '3.0', '1.4', '0.2', 'Iris-setosa'], ['5.9', '3.0', '5.1', '1.8', 'Iris-virginica'], ['6.1', '3.0', '4.6', '1.4', 'Iris-versicolor']], [['6.6', '3.0', '4.4', '1.4', 'Iris-versicolor'], ['5.2', '3.4', '1.4', '0.2', 'Iris-setosa'], ['6.9', '3.1', '5.1', '2.3', 'Iris-virginica'], ['5.8', '2.6', '4.0', '1.2', 'Iris-versicolor'], ['5.8', '2

In [156]:
# .save() testing
iris_dataset = dataset("/home/carlos/Machine_Learning_Practice/datasets/iris.data", 'last', False)
save_test = "save_test"
iris_dataset.save(save_test, "/home/carlos/Machine_Learning_Practice")

CSV file saved to /home/carlos/Machine_Learning_Practice/processed_data/save_test


In [157]:
# .extract() testing
iris_dataset = dataset('/home/carlos/Machine_Learning_Practice/processed_data/save_test.csv', 'last', True)

In [158]:
# .imputate() testing
cancer_dataset = dataset('/home/carlos/Machine_Learning_Practice/datasets/breast-cancer-wisconsin.data', 'last', False)
print(f"Partition size (estimate): {len(cancer_dataset.partitions[0])}")
print(f"Cancer Dataset w/o imputation: {cancer_dataset.partitions}")
cancer_dataset.imputate(False)
print(f"Cancer Dataset w/  imputation: {cancer_dataset.partitions}")
cancer_dataset.remove_attribute(0)
print(f"Cancer Dataset w/ first indice removed: {cancer_dataset.partitions}")

print()

votes_dataset = dataset('/home/carlos/Machine_Learning_Practice/datasets/house-votes-84.data', 'first', False)
print(f"voter Dataset w/o imputation: {votes_dataset.partitions}")
votes_dataset.imputate(True)
print(f"voter Dataset w/  imputation: {votes_dataset.partitions}")

print(type(cancer_dataset.partitions[0][0]))
print(type(cancer_dataset.partitions[0][0][0]))


Partition size (estimate): 70
Cancer Dataset w/o imputation: [[['1225382', '6', '2', '3', '1', '2', '1', '1', '1', '1', '2'], ['888820', '5', '10', '10', '3', '7', '3', '8', '10', '2', '4'], ['1365328', '1', '1', '2', '1', '2', '1', '2', '1', '1', '2'], ['706426', '5', '5', '5', '2', '5', '10', '4', '3', '1', '4'], ['144888', '8', '10', '10', '8', '5', '10', '7', '8', '1', '4'], ['1241232', '3', '1', '4', '1', '2', '?', '3', '1', '1', '2'], ['1216947', '1', '1', '1', '1', '2', '1', '3', '1', '1', '2'], ['1255384', '3', '2', '2', '3', '2', '3', '3', '1', '1', '2'], ['1080185', '10', '10', '10', '8', '6', '1', '8', '9', '1', '4'], ['1321931', '5', '1', '1', '1', '2', '1', '2', '1', '1', '2'], ['1066373', '3', '2', '1', '1', '1', '1', '2', '1', '1', '2'], ['1203096', '1', '1', '1', '1', '1', '1', '3', '1', '1', '2'], ['1193210', '2', '1', '1', '1', '2', '1', '3', '1', '1', '2'], ['1260659', '3', '1', '4', '1', '2', '1', '1', '1', '1', '2'], ['536708', '1', '1', '1', '1', '2', '1', '1', '1

In [159]:
# .discretize() testing


In [160]:
# .add_noise() testing
